In [ ]:
import tensorflow as tf
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)

In [ ]:
import keras
import pandas as pd
import tensorflow as tf

import numpy as np
# from keras.models import Sequential, Model
# from keras.layers import Dense, Dropout, Activation, Flatten, Convolution2D, MaxPooling2D, GlobalAveragePooling2D
from keras.utils import np_utils

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Convolution2D, MaxPooling2D, GlobalAveragePooling2D, Input,AveragePooling2D, BatchNormalization, LeakyReLU, SpatialDropout2D

# from keras.applications import MobileNet
# from keras.optimizers import Adam
from tensorflow.keras.applications import MobileNetV2, InceptionV3
from tensorflow.keras.optimizers import Adam

from matplotlib import pyplot as plt
import os
from sklearn.model_selection import train_test_split

import seaborn as sns
from sklearn.metrics import recall_score,auc
from sklearn.metrics import precision_score
from sklearn.model_selection import KFold

from collections import defaultdict,Counter
from imblearn.over_sampling import RandomOverSampler
from PIL import Image

from kerastuner import HyperModel

<h2>Data Loading and Preparation</h2>

In [ ]:
def getSpeciesCode(x):
    part = x.split('_')
    if part[0] == 'ASETRI':
        return 0
    elif part[0] == 'EPTBOT':
        return 1
    elif part[0] == 'MYOEMA':
        return 2
    elif part[0] == 'PIPKUH':
        return 3
    elif part[0] == 'RHIMUS':
        return 4
    elif part[0] == 'RHYNAS':
        return 5
    elif part[0] == 'ROUAEG':
        return 6
    elif part[0] == 'TAPPER':
        return 7
    else:
        return 'Unknown'
    
def getSpecies(x):
    if x == 0:
        return 'A.tridens'
    elif x == 1:
        return 'E.bottae'
    elif x == 2:
        return 'M.emarginatus'
    elif x == 3:
        return 'P.kuhli'
    elif x == 4:
        return 'R.muscatellum'
    elif x == 5:
        return 'R.nasutus'
    elif x == 6:
        return 'R.aegyptius'
    elif x == 7:
        return 'T.perforatus'
    else:
        return 'Unknown'
    
    
def generate_actual_predicted(Y_pred, X_test, Y_test): 
    predicted = list()
    for i in range(len(Y_pred)):
        predicted.append(np.argmax(Y_pred[i]))
        
    actual = list()
    for i in range(len(Y_test)):
        actual.append(np.argmax(Y_test[i]))
        
    return actual, predicted



# Ytrain in Onehot encoded form 
def makeRandomOverSamples(X_train,Y_train):
    
    Y_train_labelled=[]
    X_dims=X_train.shape

    for i in range(len(Y_train)):
        Y_train_labelled.append(np.argmax(Y_train[i]))

    print('Original training set shape %s' %  [(getSpecies(k),v) for k,v in Counter(Y_train_labelled).items()])       
    X_train= X_train.reshape(-1,X_dims[1]*X_dims[2]*X_dims[3])

    ros = RandomOverSampler(random_state=123)
    X_train, Y_train_labelled = ros.fit_resample(X_train, Y_train_labelled)


    print('Resampled training set shape %s' % [(getSpecies(k),v) for k,v in Counter(Y_train_labelled).items()])


    #reshape X_all
    X_train= X_train.reshape(-1,X_dims[1],X_dims[2],X_dims[3])

    # update Y_train
    Y_train= np_utils.to_categorical(Y_train_labelled, num_classes=8)

    print("After OverSampling\nX_train: shape= ",X_train.shape)
    print("Y_train: shape= ",Y_train.shape)

    
    
    return(X_train,Y_train)

In [ ]:
image_folder_path = '../../data/MFCC2'
file_paths = [f for f in os.listdir(image_folder_path)]
species = []
for file_name in file_paths:
    sp = file_name.split('_')
    species.append(sp[0])

df = pd.DataFrame(species, columns=['Species'])
df['Species'].value_counts()

In [ ]:
X_all = []
Y_all = []

image_folder_path = '../../data/MFCC2'
file_paths = [f for f in os.listdir(image_folder_path)]

for file_name in file_paths:
    spectrogram = Image.open(image_folder_path + '/' + file_name)
    spectrogram = spectrogram.convert('RGB')
    spectrogram = spectrogram.resize((170, 112))  
    spectrogram = np.array(spectrogram)
    #spectrogram = np.expand_dims(spectrogram, axis=2) 
    X_all.append(spectrogram)
    Y_all.append(getSpeciesCode(file_name))


In [ ]:
X_all = np.array(X_all)
Y_all = np.array(Y_all)

print(X_all.shape)
print(Y_all.shape)

X_all = X_all.astype('float32')
X_all /= 255

Y_all = np_utils.to_categorical(Y_all, num_classes=8) 


# Evaluation: Utility Functions

In [ ]:
# onehot encoded Y_all
def kFold_train(X_all,Y_all, mode_version):
    from sklearn.model_selection import StratifiedKFold
    n_split=10
    n_classes=8

    y_all_labelled=[]
    for i in range(len(Y_all)):
            y_all_labelled.append(np.argmax(Y_all[i]))

    my_callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=20,verbose=1, mode='min',restore_best_weights=True)]

    count = 1

    for train_index,test_index in StratifiedKFold(n_split, shuffle=True, random_state=123).split(X_all,y_all_labelled):  
        # use the index to generate training an testing sets
        x_train,x_test=X_all[train_index],X_all[test_index]
        y_train,y_test=Y_all[train_index],Y_all[test_index]
        
        
       
        x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state = 245, stratify=y_train)
        x_train, y_train= makeRandomOverSamples(x_train, y_train)

        # create and fit the model
        cv_model=build_current_model() # lr,dense,cnn
        history = cv_model.fit(x_train, y_train,
                  batch_size=8,
                  epochs=100,
                  validation_data= (x_val, y_val),
                  verbose=2,
                  callbacks=my_callbacks)


        file_name = mode_version +'_iter_' +str(count)
        
        
                
        #store test index
#         csv_name= mode_version +'_testset_iter_' +str(count)
#         test_index_df = pd.DataFrame(data=test_index,columns=["test_index"])
#         with open(csv_name, mode='w') as f:
#                  test_index_df.to_csv(f)
        #Store History
        hist_df = pd.DataFrame(history.history) 
        hist_json_file = file_name + '.json'

        with open(hist_json_file, mode='w') as f:
            hist_df.to_json(f)
            
            

        model_file = file_name + '.model'
        print(model_file)

        #Save Model
        cv_model.save(model_file) #Save the model

        count+=1

In [ ]:
def kFold_test(X_all,Y_all, mode_version):
    from sklearn.metrics import precision_recall_fscore_support
    from sklearn.model_selection import StratifiedKFold
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, auc, precision_recall_curve
    from sklearn.metrics import confusion_matrix

    n_split=10
    n_classes=8

    all_base_precision = list()
    all_base_recall = list()
    all_macro_precision = list()
    all_macro_recall = list()
    all_f1 = list()
    all_macro_f1 = list()

    all_tpr = list()
    all_fpr = list()
    all_precision = list()
    all_recall = list()
    all_cm=list()
    all_accuracy = list()

    all_df = list()

    y_all_labelled=[]
    for i in range(len(Y_all)):
            y_all_labelled.append(np.argmax(Y_all[i]))

    count = 1


    for train_index,test_index in StratifiedKFold(n_split, shuffle=True, random_state=123).split(X_all,y_all_labelled):  
        # use the index to generate training an testing sets
        x_train,_=X_all[train_index],X_all[test_index]
        y_train,y_test=Y_all[train_index],Y_all[test_index]


        file_name = mode_version +'_iter_' +str(count)
#         csv_name= mode_version +'_testset_iter_' +str(count)
        #Load History 
        hist_json_file = file_name + '.json'
        history_info = pd.read_json(hist_json_file)
        all_df.append(history_info)


        # Load Model
        model_file = file_name + '.model'
        print('File Name Loaded: ',model_file)
        cv_model = tf.keras.models.load_model(model_file) 
        
#         #load test set 
#         testset_file= csv_name
#         testset_df= pd.read_csv (testset_file)
#         test_index=testset_df["test_index"].to_numpy()
       
#         x_test=X_all[test_index]
#         y_test=Y_all[test_index]


        count+=1

        #generate predictions
        y_pred = cv_model.predict(x_test)
        actual, predicted = generate_actual_predicted(y_pred, x_test, y_test)

        #calc metrics
        curr_base_prec, curr_base_rec, curr_f1, _ = precision_recall_fscore_support(actual, predicted)
        curr_macro_prec = precision_score(actual, predicted,average='macro')
        curr_macro_rec = recall_score(actual, predicted,average='macro')
        curr_macro_f1 = f1_score(actual, predicted,average='macro')

        actual_labeled = list()
        predict_labeled = list()
        for x,y in zip(actual,predicted):
            actual_labeled.append(getSpecies(x))
            predict_labeled.append(getSpecies(y))

        labels = ['A.tridens','E.bottae','M.emarginatus','P.kuhli','R.muscatellum','R.nasutus', 'R.aegyptius', 'T.perforatus']
        curr_cm=confusion_matrix(actual_labeled,predict_labeled,labels=labels)

        curr_fpr = [0] * n_classes
        curr_tpr = [0] * n_classes
        for i in range(n_classes):
            curr_fpr[i], curr_tpr[i], _ = roc_curve(y_test[:,i], y_pred[:,i])

        curr_prec = [0] * n_classes
        curr_rec = [0] * n_classes
        for i in range(n_classes):
            curr_prec[i], curr_rec[i], _ = precision_recall_curve(y_test[:,i], y_pred[:,i])


        curr_accuracy = accuracy_score(actual, predicted)

        #add to lists
        all_base_precision.append(curr_base_prec)
        all_base_recall.append(curr_base_rec)
        all_macro_precision.append(curr_macro_prec)
        all_macro_recall.append(curr_macro_rec)

        all_f1.append(curr_f1)
        all_macro_f1.append(curr_macro_f1)

        all_accuracy.append(curr_accuracy)


        all_fpr.append(curr_fpr)
        all_tpr.append(curr_tpr)
        all_precision.append(curr_prec)
        all_recall.append(curr_rec)
        all_cm.append(curr_cm)
    kFold_metrics={
            "all_base_precision": all_base_precision ,
            "all_base_recall": all_base_recall,
            "all_macro_precision":all_macro_precision ,
            "all_macro_recall":all_macro_recall ,
            "all_f1":all_f1,
            "all_macro_f1":all_macro_f1 ,
        
            "all_accuracy":all_accuracy ,
            "all_fpr":all_fpr,
            "all_tpr": all_tpr,
            "all_precision":all_precision,
            "all_recall":all_recall,
            "all_cm": all_cm,
            "all_df":all_df
            
        }
    return kFold_metrics




In [ ]:
def plot_loss_Accuracy(all_df):
    for df in all_df:
        _, axs = plt.subplots(ncols=2)

        sns.lineplot(data=df['loss'],ax=axs[0], label='Training Loss', legend='brief')
        sns.lineplot(data=df['val_loss'],ax=axs[0], label='Validation Loss', legend='brief')

        sns.lineplot(data=df['acc'],ax=axs[1], label='Training Accuracy', legend='brief')
        sns.lineplot(data=df['val_acc'],ax=axs[1], label='Validation Accuracy', legend='brief')

In [ ]:
def print_metrics(KFold_metrics):
    print('precision mean by class', np.array(KFold_metrics.get("all_base_precision")).mean(axis=0))
    print('precision sdev by class', np.array(KFold_metrics.get("all_base_precision")).std(axis=0))

    print('macro precision mean', np.array(KFold_metrics.get("all_macro_precision")).mean())
    print('macro precision sdev', np.array(KFold_metrics.get("all_macro_precision")).std())


    print('recall mean', np.array(KFold_metrics.get("all_base_recall")).mean(axis=0))
    print('recall sdev', np.array(KFold_metrics.get("all_base_recall")).std(axis=0))

    print('macro recall mean', np.array(KFold_metrics.get("all_macro_recall")).mean())
    print('macro recall sdev', np.array(KFold_metrics.get("all_macro_recall")).std())

    print('f1 mean', np.array(KFold_metrics.get("all_f1")).mean(axis=0))
    print('f1 sdev', np.array(KFold_metrics.get("all_f1")).std(axis=0))

    print('macro f1 mean', np.array(KFold_metrics.get("all_macro_f1")).mean())
    print('macro f1 sdev', np.array(KFold_metrics.get("all_macro_f1")).std())

    print('accuracy mean', np.array(KFold_metrics.get("all_accuracy")).mean())
    print('accuracy sdev', np.array(KFold_metrics.get("all_accuracy")).std())



In [ ]:
# Plot all K-Fold ROC curves
def plot_ROC(all_fpr,all_tpr,n_split=10, n_classes=8):
    for i in range(n_split):
        _, axs = plt.subplots(ncols=1)
        axs.set(xlabel='False Positive Rate',ylabel='True Positive Rate', title='ROC Curve K-Fold#'+str(i+1))
        for j in range(n_classes):
            auc_val = auc(all_fpr[i][j], all_tpr[i][j])
            auc_val = np.around(auc_val,4)
            sns.lineplot(x=all_fpr[i][j],y=all_tpr[i][j],ax=axs, label='Class '+ getSpecies(j) +' (area = ' + str(auc_val) + ')', legend='brief')


In [ ]:
def print_meanAUC(all_fpr,all_tpr,n_split=10, n_classes=8):
    aucs = list()

    for i in range(n_classes):
        aucs.append(list())


    for i in range(n_split):
        for j in range(n_classes):
            auc_val = auc(all_fpr[i][j], all_tpr[i][j])
            aucs[j].append(auc_val)

    for i in range(n_classes):
        print("Sp: ", getSpecies(i))
        print("AUC Mean ", np.array(aucs[i]).mean())
        print("Std ", np.array(aucs[i]).std())

In [ ]:
#Plot all K-Fold Precision/Recall curves
def plot_precision_recall(all_recall,all_precision,n_split=10, n_classes=8):
    for i in range(n_split):
        _, axs = plt.subplots(ncols=1)
        axs.set(xlabel='Recall',ylabel='Precision', title='Precision/Recall Curve K-Fold#'+str(i+1))
        for j in range(n_classes):
            sns.lineplot(x=all_recall[i][j],y=all_precision[i][j],ax=axs, label='Class '+ getSpecies(j), legend='brief')

In [ ]:
def plot_confusion_matrices_per_iter(all_cm,n_split=10, n_classes=8):
    ## Plot all K-Fold confusion matrices
    import numpy as np
    import matplotlib.pylab as pl
    import matplotlib.gridspec as gridspec
    from matplotlib import pyplot as plt
    labels = ['A.tridens','E.bottae','M.emarginatus','P.kuhli','R.muscatellum','R.nasutus', 'R.aegyptius', 'T.perforatus']
        
    # Create 1X 5 sub plots
    # gs = gridspec.GridSpec(1,n_split)
    figs=[]
    for i in range(n_split):
        figs.append(plt.figure())

    for i in range(n_split):
        print('Confusion Matrix K-Fold #'+ str(i+1)+"\n")
        print(all_cm[i])
        print("\n")

        ax=figs[i].add_subplot()
        sns.heatmap(all_cm[i], annot=True, ax = ax); #annot=True to annotate cells


        # labels, title and ticks
        ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels'); 
        ax.set_title('Confusion Matrix K-Fold #'+ str(i+1)); 
        ax.xaxis.set_ticklabels(labels,rotation=45); ax.yaxis.set_ticklabels(labels,rotation=45);

In [ ]:
def  plot_macro_confusion_matrices(all_cm,n_split=10, n_classes=8):
    ## plot macro cm
    labels = ['A.tridens','E.bottae','M.emarginatus','P.kuhli','R.muscatellum','R.nasutus', 'R.aegyptius', 'T.perforatus']
        
    sum_all_cm=np.zeros((n_classes,n_classes)).astype('int64')
    for i in range(n_split):
        sum_all_cm+=all_cm[i]


    #plot 
    figs=[]
    for i in range(2):
        figs.append(plt.figure())

    print("sum of all confuion matrices\n",sum_all_cm)
    ax=figs[0].add_subplot()
    sns.heatmap(sum_all_cm, annot=True, ax = ax);

    avg_all_cm=np.divide(sum_all_cm,n_split).astype('int64')
    print("\naverage of all confuion matrices\n",avg_all_cm)
    ax=figs[1].add_subplot()
    sns.heatmap(avg_all_cm, annot=True, ax = ax);
    return sum_all_cm, avg_all_cm

In [ ]:
 def plot_normalised_confusion_matrix( sum_all_cm, avg_all_cm, n_split=10, n_classes=8):
    import numpy as np
    import matplotlib.pyplot as plt
    import seaborn as sns
    labels = ['A.tridens','E.bottae','M.emarginatus','P.kuhli','R.muscatellum','R.nasutus', 'R.aegyptius', 'T.perforatus']
        

    cm= sum_all_cm/ sum_all_cm.astype(np.float).sum(axis=1,keepdims = True)
    cm = (np.around(cm,2))
    print(cm)


    labels = ['A.tridens','E.bottae','M.emarginatus','P.kuhli','R.muscatellum','R.nasutus', 'R.aegyptius', 'T.perforatus']


    ax= plt.subplot()
    sns.heatmap(cm, annot=True, ax = ax); #annot=True to annotate cells

    # labels, title and ticks
    ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels'); 
    ax.set_title('Confusion Matrix'); 
    ax.xaxis.set_ticklabels(labels,rotation=45); ax.yaxis.set_ticklabels(labels,rotation=45);

### Load and Evaluate



## V1 !!!!! K-Fold Training and Evaluation


In [8]:
folder = "models"
mode_version = "spec_RO_v1"
mode_version = os.path.join(folder, mode_version)

print(mode_version)

models/spec_smote_v1


In [9]:
def build_current_model():
    inp = Input(shape=(112,170,3))
    lay = Convolution2D(filters=16,kernel_size=(7,7),strides=(2,2),padding='same')(inp)
    lay = BatchNormalization()(lay)
    lay = Activation('relu')(lay)
    lay = MaxPooling2D(pool_size=(2,2),strides=2,padding='same')(lay)
    lay = SpatialDropout2D(0.06)(lay)

    lay = Convolution2D(filters=64,kernel_size=(5,5),strides=(2,2),padding='same')(lay)
    lay = BatchNormalization()(lay)
    lay = Activation('relu')(lay)
    lay = AveragePooling2D(pool_size=(2,2),strides=2,padding='same')(lay)
    lay = SpatialDropout2D(0.06)(lay)

    lay = Convolution2D(filters=112,kernel_size=(3,3),strides=(1,1),padding='same')(lay)
    lay = BatchNormalization()(lay)
    lay = Activation('relu')(lay)
    lay = AveragePooling2D(pool_size=(2,2),strides=2,padding='same')(lay)
    lay = SpatialDropout2D(0.02)(lay)

    lay = Convolution2D(filters=64,kernel_size=(3,3),strides=(1,1),padding='same')(lay)
    lay = BatchNormalization()(lay)
    lay = Activation('relu')(lay)
    lay = AveragePooling2D(pool_size=(2,2),strides=2,padding='same')(lay)
    lay = SpatialDropout2D(0.02)(lay)

    lay = Flatten()(lay)

    lay = Dense(80)(lay)
    lay = BatchNormalization()(lay)
    lay = Activation('relu')(lay)
    lay = Dropout(0.15)(lay)
    
    lay = Dense(96)(lay)
    lay = BatchNormalization()(lay)
    lay = Activation('relu')(lay)
    lay = Dropout(0.15)(lay)
    
    lay = Dense(32)(lay)
    lay = BatchNormalization()(lay)
    lay = Activation('relu')(lay)
    lay = Dropout(0.15)(lay)

    x_out = Dense(8, name='output', activation='softmax')(lay)
    model = Model(inputs=inp, outputs=x_out)
    model.compile(
        optimizer=Adam(lr=0.003),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model


In [ ]:
model = build_current_model()
model.summary()

In [ ]:
kFold_train(X_all,Y_all, mode_version)
               

In [ ]:
kFold_metrics=kFold_test(X_all,Y_all, mode_version)

In [ ]:
# all_df=[]
# for i in range(10):
    
#      #Load History 
#         hist_json_file =  mode_version +'_iter_' +str(i+1) + '.json'
#         history_info = pd.read_json(hist_json_file)
#         all_df.append(history_info)
# print (all_df[99])
# plot_loss_Accuracy(all_df)

In [ ]:
# print(KFold_metrics.get("all_df"))
plot_loss_Accuracy(kFold_metrics.get("all_df"))

In [ ]:
print_metrics(kFold_metrics)


In [ ]:
plot_ROC(kFold_metrics.get("all_fpr"),kFold_metrics.get("all_tpr"))


In [ ]:
print_meanAUC (kFold_metrics.get("all_fpr"),kFold_metrics.get("all_tpr"))


In [ ]:
plot_precision_recall(kFold_metrics.get("all_recall"),kFold_metrics.get("all_precision"))
                   

In [ ]:
plot_confusion_matrices_per_iter(kFold_metrics.get("all_cm"))


In [ ]:
sum_all_cm, avg_all_cm=plot_macro_confusion_matrices(kFold_metrics.get("all_cm"))                   


In [ ]:
plot_normalised_confusion_matrix(sum_all_cm, avg_all_cm)

<h2> V2 !!!!! K-Fold Training and Evaluation</h2>

In [8]:
folder = "models"
mode_version = "spec_RO_v2"
mode_version = os.path.join(folder, mode_version)

print(mode_version)

models/spec_smote_v1


In [31]:
def build_current_model():
    inp = Input(shape=(112,170,3))
    lay = Convolution2D(filters=24,kernel_size=(5,5),strides=(2,2),padding='same')(inp)
    lay = BatchNormalization()(lay)
    lay = Activation('relu')(lay)
    lay = MaxPooling2D(pool_size=(2,2),strides=2,padding='same')(lay)
    lay = SpatialDropout2D(0.06)(lay)

    lay = Convolution2D(filters=56,kernel_size=(5,5),strides=(2,2),padding='same')(lay)
    lay = BatchNormalization()(lay)
    lay = Activation('relu')(lay)
    lay = AveragePooling2D(pool_size=(2,2),strides=2,padding='same')(lay)
    lay = SpatialDropout2D(0.06)(lay)

    lay = Convolution2D(filters=136,kernel_size=(7,7),strides=(1,1),padding='same')(lay)
    lay = BatchNormalization()(lay)
    lay = Activation('relu')(lay)
    lay = AveragePooling2D(pool_size=(2,2),strides=2,padding='same')(lay)
    lay = SpatialDropout2D(0.02)(lay)

    lay = Convolution2D(filters=120,kernel_size=(5,5),strides=(1,1),padding='same')(lay)
    lay = BatchNormalization()(lay)
    lay = Activation('relu')(lay)
    lay = AveragePooling2D(pool_size=(2,2),strides=2,padding='same')(lay)
    lay = SpatialDropout2D(0.02)(lay)

    lay = Flatten()(lay)

    lay = Dense(512)(lay)
    lay = BatchNormalization()(lay)
    lay = Activation('relu')(lay)
    lay = Dropout(0.4)(lay)

    lay = Dense(256)(lay)
    lay = BatchNormalization()(lay)
    lay = Activation('relu')(lay)
    lay = Dropout(0.3)(lay)

    lay = Dense(128)(lay)
    lay = BatchNormalization()(lay)
    lay = Activation('relu')(lay)
    lay = Dropout(0.2)(lay)

    x_out = Dense(8, name='output', activation='softmax')(lay)
    model = Model(inputs=inp, outputs=x_out)
    model.compile(
        optimizer=Adam(lr=0.003),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model


In [ ]:
model = build_current_model()
model.summary()


In [ ]:
kFold_train(X_all,Y_all, mode_version)
               

In [ ]:
kFold_metrics=kFold_test(X_all,Y_all, mode_version)

In [ ]:
# print(KFold_metrics.get("all_df"))
plot_loss_Accuracy(kFold_metrics.get("all_df"))

In [ ]:
print_metrics(kFold_metrics)


In [ ]:
plot_ROC(kFold_metrics.get("all_fpr"),kFold_metrics.get("all_tpr"))


In [ ]:
print_meanAUC (kFold_metrics.get("all_fpr"),kFold_metrics.get("all_tpr"))


In [ ]:
plot_precision_recall(kFold_metrics.get("all_recall"),kFold_metrics.get("all_precision"))
                   

In [ ]:
plot_confusion_matrices_per_iter(kFold_metrics.get("all_cm"))


In [ ]:
sum_all_cm, avg_all_cm=plot_macro_confusion_matrices(kFold_metrics.get("all_cm"))                   


In [ ]:
plot_normalised_confusion_matrix(sum_all_cm, avg_all_cm)

<h2> V3 !!!!! K-Fold Training and Evaluation</h2>

In [ ]:
folder = "models"
mode_version = "spec_RO_v3"
mode_version = os.path.join(folder, mode_version)

print(mode_version)

In [13]:
def build_current_model():
    inp = Input(shape=(112,170,3))
    lay = Convolution2D(filters=176,kernel_size=(7,7),strides=(2,2),padding='same')(inp)
    lay = BatchNormalization()(lay)
    lay = Activation('relu')(lay)
    lay = MaxPooling2D(pool_size=(2,2),strides=2,padding='same')(lay)
    lay = SpatialDropout2D(0.06)(lay)

    lay = Convolution2D(filters=48,kernel_size=(5,5),strides=(2,2),padding='same')(lay)
    lay = BatchNormalization()(lay)
    lay = Activation('relu')(lay)
    lay = AveragePooling2D(pool_size=(2,2),strides=2,padding='same')(lay)
    lay = SpatialDropout2D(0.06)(lay)

    lay = Convolution2D(filters=144,kernel_size=(3,3),strides=(1,1),padding='same')(lay)
    lay = BatchNormalization()(lay)
    lay = Activation('relu')(lay)
    lay = AveragePooling2D(pool_size=(2,2),strides=2,padding='same')(lay)
    lay = SpatialDropout2D(0.02)(lay)

    lay = Convolution2D(filters=192,kernel_size=(3,3),strides=(1,1),padding='same')(lay)
    lay = BatchNormalization()(lay)
    lay = Activation('relu')(lay)
    lay = AveragePooling2D(pool_size=(2,2),strides=2,padding='same')(lay)
    lay = SpatialDropout2D(0.02)(lay)

    lay = Flatten()(lay)

    lay = Dense(144)(lay)
    lay = BatchNormalization()(lay)
    lay = Activation('relu')(lay)
    lay = Dropout(0.15)(lay)
    
    lay = Dense(80)(lay)
    lay = BatchNormalization()(lay)
    lay = Activation('relu')(lay)
    lay = Dropout(0.15)(lay)
    
    lay = Dense(96)(lay)
    lay = BatchNormalization()(lay)
    lay = Activation('relu')(lay)
    lay = Dropout(0.15)(lay)

    x_out = Dense(8, name='output', activation='softmax')(lay)
    model = Model(inputs=inp, outputs=x_out)
    model.compile(
        optimizer=Adam(lr=0.003),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model


In [ ]:
model = build_current_model()
model.summary()

In [ ]:
kFold_train(X_all,Y_all, mode_version)
               

In [ ]:
kFold_metrics=kFold_test(X_all,Y_all, mode_version)

In [ ]:
# print(KFold_metrics.get("all_df"))
plot_loss_Accuracy(kFold_metrics.get("all_df"))

In [ ]:
print_metrics(kFold_metrics)


In [ ]:
plot_ROC(kFold_metrics.get("all_fpr"),kFold_metrics.get("all_tpr"))


In [ ]:
print_meanAUC (kFold_metrics.get("all_fpr"),kFold_metrics.get("all_tpr"))


In [ ]:
plot_precision_recall(kFold_metrics.get("all_recall"),kFold_metrics.get("all_precision"))
                   

In [ ]:
plot_confusion_matrices_per_iter(kFold_metrics.get("all_cm"))


In [ ]:
sum_all_cm, avg_all_cm=plot_macro_confusion_matrices(kFold_metrics.get("all_cm"))                   


In [ ]:
plot_normalised_confusion_matrix(sum_all_cm, avg_all_cm)

<h2> V4 !!!!! K-Fold Training and Evaluation</h2>

In [ ]:
folder = "models"
mode_version = "spec_RO_v4"
mode_version = os.path.join(folder, mode_version)

print(mode_version)

In [35]:
def build_current_model():
    inp = Input(shape=(112,170,3))
    lay = Convolution2D(filters=160,kernel_size=(7,7),strides=(2,2),padding='same')(inp)
    lay = BatchNormalization()(lay)
    lay = Activation('relu')(lay)
    lay = MaxPooling2D(pool_size=(2,2),strides=2,padding='same')(lay)
    lay = SpatialDropout2D(0.06)(lay)

    lay = Convolution2D(filters=112,kernel_size=(5,5),strides=(2,2),padding='same')(lay)
    lay = BatchNormalization()(lay)
    lay = Activation('relu')(lay)
    lay = AveragePooling2D(pool_size=(2,2),strides=2,padding='same')(lay)
    lay = SpatialDropout2D(0.06)(lay)

    lay = Convolution2D(filters=80,kernel_size=(3,3),strides=(1,1),padding='same')(lay)
    lay = BatchNormalization()(lay)
    lay = Activation('relu')(lay)
    lay = AveragePooling2D(pool_size=(2,2),strides=2,padding='same')(lay)
    lay = SpatialDropout2D(0.02)(lay)

    lay = Convolution2D(filters=64,kernel_size=(3,3),strides=(1,1),padding='same')(lay)
    lay = BatchNormalization()(lay)
    lay = Activation('relu')(lay)
    lay = AveragePooling2D(pool_size=(2,2),strides=2,padding='same')(lay)
    lay = SpatialDropout2D(0.02)(lay)

    lay = Flatten()(lay)

    lay = Dense(256)(lay)
    lay = BatchNormalization()(lay)
    lay = Activation('relu')(lay)
    lay = Dropout(0.15)(lay)
    
    lay = Dense(448)(lay)
    lay = BatchNormalization()(lay)
    lay = Activation('relu')(lay)
    lay = Dropout(0.15)(lay)
    
    lay = Dense(352)(lay)
    lay = BatchNormalization()(lay)
    lay = Activation('relu')(lay)
    lay = Dropout(0.15)(lay)

    x_out = Dense(8, name='output', activation='softmax')(lay)
    model = Model(inputs=inp, outputs=x_out)
    model.compile(
        optimizer=Adam(lr=0.003),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model



In [ ]:
model = build_current_model()
model.summary()

In [ ]:
kFold_train(X_all,Y_all, mode_version)
               

In [ ]:
kFold_metrics=kFold_test(X_all,Y_all, mode_version)

In [ ]:
# print(KFold_metrics.get("all_df"))
plot_loss_Accuracy(kFold_metrics.get("all_df"))

In [ ]:
print_metrics(kFold_metrics)


In [ ]:
plot_ROC(kFold_metrics.get("all_fpr"),kFold_metrics.get("all_tpr"))


In [ ]:
print_meanAUC (kFold_metrics.get("all_fpr"),kFold_metrics.get("all_tpr"))


In [ ]:
plot_precision_recall(kFold_metrics.get("all_recall"),kFold_metrics.get("all_precision"))
                   

In [ ]:
plot_confusion_matrices_per_iter(kFold_metrics.get("all_cm"))


In [ ]:
sum_all_cm, avg_all_cm=plot_macro_confusion_matrices(kFold_metrics.get("all_cm"))                   


In [ ]:
plot_normalised_confusion_matrix(sum_all_cm, avg_all_cm)

<h2> V5 !!!!! K-Fold Training and Evaluation</h2>

In [ ]:
folder = "models"
mode_version = "spec_RO_v5"
mode_version = os.path.join(folder, mode_version)

print(mode_version)

In [49]:
def build_current_model():
    inp = Input(shape=(112,170,3))
    lay = Convolution2D(filters=48,kernel_size=(7,7),strides=(2,2),padding='same')(inp)
    lay = BatchNormalization()(lay)
    lay = Activation('relu')(lay)
    lay = MaxPooling2D(pool_size=(2,2),strides=2,padding='same')(lay)
    lay = SpatialDropout2D(0.06)(lay)

    lay = Convolution2D(filters=32,kernel_size=(5,5),strides=(2,2),padding='same')(lay)
    lay = BatchNormalization()(lay)
    lay = Activation('relu')(lay)
    lay = AveragePooling2D(pool_size=(2,2),strides=2,padding='same')(lay)
    lay = SpatialDropout2D(0.06)(lay)

    lay = Convolution2D(filters=48,kernel_size=(3,3),strides=(1,1),padding='same')(lay)
    lay = BatchNormalization()(lay)
    lay = Activation('relu')(lay)
    lay = AveragePooling2D(pool_size=(2,2),strides=2,padding='same')(lay)
    lay = SpatialDropout2D(0.02)(lay)

    lay = Convolution2D(filters=128,kernel_size=(3,3),strides=(1,1),padding='same')(lay)
    lay = BatchNormalization()(lay)
    lay = Activation('relu')(lay)
    lay = AveragePooling2D(pool_size=(2,2),strides=2,padding='same')(lay)
    lay = SpatialDropout2D(0.02)(lay)

    lay = Flatten()(lay)

    lay = Dense(112)(lay)
    lay = BatchNormalization()(lay)
    lay = Activation('relu')(lay)
    lay = Dropout(0.15)(lay)
    
    lay = Dense(32)(lay)
    lay = BatchNormalization()(lay)
    lay = Activation('relu')(lay)
    lay = Dropout(0.15)(lay)

    x_out = Dense(8, name='output', activation='softmax')(lay)
    model = Model(inputs=inp, outputs=x_out)
    model.compile(
        optimizer=Adam(lr=0.003),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model


In [ ]:
model = build_current_model()
model.summary()

In [ ]:
kFold_train(X_all,Y_all, mode_version)
               

In [ ]:
kFold_metrics=kFold_test(X_all,Y_all, mode_version)

In [ ]:
# print(KFold_metrics.get("all_df"))
plot_loss_Accuracy(kFold_metrics.get("all_df"))

In [ ]:
print_metrics(kFold_metrics)


In [ ]:
plot_ROC(kFold_metrics.get("all_fpr"),kFold_metrics.get("all_tpr"))


In [ ]:
print_meanAUC (kFold_metrics.get("all_fpr"),kFold_metrics.get("all_tpr"))


In [ ]:
plot_precision_recall(kFold_metrics.get("all_recall"),kFold_metrics.get("all_precision"))
                   

In [ ]:
plot_confusion_matrices_per_iter(kFold_metrics.get("all_cm"))


In [ ]:
sum_all_cm, avg_all_cm=plot_macro_confusion_matrices(kFold_metrics.get("all_cm"))                   


In [ ]:
plot_normalised_confusion_matrix(sum_all_cm, avg_all_cm)

# V6 Re-Load Data

In [ ]:
folder = "models"
mode_version = "spec_RO_v6"
mode_version = os.path.join(folder, mode_version)

print(mode_version)

In [63]:
X_all = []
Y_all = []

image_folder_path = '../../data/MFCC2'
file_paths = [f for f in os.listdir(image_folder_path)]

for file_name in file_paths:
    spectrogram = Image.open(image_folder_path + '/' + file_name)
    spectrogram = spectrogram.convert('RGB')
    spectrogram = spectrogram.resize((85, 56))  
    spectrogram = np.array(spectrogram)
    #spectrogram = np.expand_dims(spectrogram, axis=2) 
    X_all.append(spectrogram)
    Y_all.append(getSpeciesCode(file_name))


In [64]:
X_all = np.array(X_all)
Y_all = np.array(Y_all)

print(X_all.shape)
print(Y_all.shape)

X_all = X_all.astype('float32')
X_all /= 255

Y_all = np_utils.to_categorical(Y_all, num_classes=8) 


(3018, 56, 85, 3)
(3018,)


In [65]:
def build_current_model():
    inp = Input(shape=(56,85,3))
    lay = Convolution2D(filters=168,kernel_size=(7,7),strides=(2,2),padding='same')(inp)
    lay = BatchNormalization()(lay)
    lay = Activation('relu')(lay)
    lay = MaxPooling2D(pool_size=(2,2),strides=2,padding='same')(lay)
    lay = Dropout(0.2)(lay)

    lay = Convolution2D(filters=56,kernel_size=(7,7),strides=(2,2),padding='same')(lay)
    lay = BatchNormalization()(lay)
    lay = Activation('relu')(lay)
    lay = AveragePooling2D(pool_size=(2,2),strides=2,padding='same')(lay)
    lay = Dropout(0.3)(lay)

    lay = Convolution2D(filters=232,kernel_size=(5,5),strides=(1,1),padding='same')(lay)
    lay = BatchNormalization()(lay)
    lay = Activation('relu')(lay)
    lay = AveragePooling2D(pool_size=(2,2),strides=2,padding='same')(lay)
    lay = Dropout(0.3)(lay)

    lay = Convolution2D(filters=104,kernel_size=(5,5),strides=(1,1),padding='same')(lay)
    lay = BatchNormalization()(lay)
    lay = Activation('relu')(lay)
    lay = AveragePooling2D(pool_size=(2,2),strides=2,padding='same')(lay)
    lay = Dropout(0.4)(lay)

    lay = Flatten()(lay)

    x_out = Dense(8, name='output', activation='softmax')(lay)
    model = Model(inputs=inp, outputs=x_out)
    model.compile(
        optimizer=Adam(lr=0.003),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model



In [ ]:
model = build_current_model()
model.summary()

In [ ]:
kFold_train(X_all,Y_all, mode_version)
               

In [ ]:
kFold_metrics=kFold_test(X_all,Y_all, mode_version)

In [ ]:
# print(KFold_metrics.get("all_df"))
plot_loss_Accuracy(kFold_metrics.get("all_df"))

In [ ]:
print_metrics(kFold_metrics)


In [ ]:
plot_ROC(kFold_metrics.get("all_fpr"),kFold_metrics.get("all_tpr"))


In [ ]:
print_meanAUC (kFold_metrics.get("all_fpr"),kFold_metrics.get("all_tpr"))


In [ ]:
plot_precision_recall(kFold_metrics.get("all_recall"),kFold_metrics.get("all_precision"))
                   

In [ ]:
plot_confusion_matrices_per_iter(kFold_metrics.get("all_cm"))


In [ ]:
sum_all_cm, avg_all_cm=plot_macro_confusion_matrices(kFold_metrics.get("all_cm"))                   


In [ ]:
plot_normalised_confusion_matrix(sum_all_cm, avg_all_cm)